In [ ]:
import os
import cv2
import face_recognition
import numpy as np
import faiss

# Initialize FAISS index (FlatL2 for simplicity)
dimension = 128  # Face encoding dimension
index = faiss.IndexFlatL2(dimension)

# Database to store user info
user_db = {}

def register_user_with_webcam(empid, name):
    # Create a folder for the user using empid
    user_folder = os.path.join("users", empid)
    if not os.path.exists(user_folder):
        os.makedirs(user_folder)

    # Capture image from the webcam
    cam = cv2.VideoCapture(0)
    cv2.namedWindow("Register")

    print("Press SPACE to capture the image and ESC to exit.")
    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to grab frame")
            break
        cv2.imshow("Register", frame)

        k = cv2.waitKey(1)
        if k % 256 == 27:  # ESC pressed
            print("Escape hit, closing...")
            break
        elif k % 256 == 32:  # SPACE pressed
            img_name = os.path.join(user_folder, f"{empid}.jpg")
            cv2.imwrite(img_name, frame)
            print(f"{img_name} saved!")
            break

    cam.release()
    cv2.destroyAllWindows()

    # Load the image and convert it to RGB
    image = cv2.imread(img_name)
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect face and encode
    face_locations = face_recognition.face_locations(rgb_image)
    face_encodings = face_recognition.face_encodings(rgb_image, face_locations)

    if len(face_encodings) > 0:
        face_encoding = face_encodings[0]
        # Store in FAISS
        index.add(np.array([face_encoding]))
        # Map empid to index and store user info
        user_db[empid] = {
            "index": index.ntotal - 1,
            "name": name,
            "folder": user_folder  # Store user folder path for later verification
        }
        print(f"User {name} (empid: {empid}) registered successfully.")
    else:
        print("No face found in the image.")

# Example usage for registration
if __name__ == "__main__":
    empid = input("Enter your employee ID: ")
    name = input("Enter your name: ")
    register_user_with_webcam(empid, name)


In [8]:
#Verify

import cv2
import face_recognition
import numpy as np
import faiss

# Initialize FAISS index (FlatL2 for simplicity)
dimension = 128  # Face encoding dimension
index = faiss.IndexFlatL2(dimension)

# Database to store user info
user_db = {}  # Ideally, load this from a persistent storage like a database

def load_user_database():
    # Simulated function to load user database from a persistent storage
    pass

def verify_user_with_webcam():
    load_user_database()  # Load user data into user_db (simulated)

    cam = cv2.VideoCapture(0)
    cv2.namedWindow("Verify")

    print("Press SPACE to capture the image for verification and ESC to exit.")
    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to grab frame")
            break
        cv2.imshow("Verify", frame)

        k = cv2.waitKey(1)
        if k % 256 == 27:  # ESC pressed
            print("Escape hit, closing...")
            break
        elif k % 256 == 32:  # SPACE pressed
            # Load the image and convert it to RGB
            rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Detect face and encode
            face_locations = face_recognition.face_locations(rgb_image)
            face_encodings = face_recognition.face_encodings(rgb_image, face_locations)

            if len(face_encodings) > 0:
                face_encoding = face_encodings[0]

                # Search in FAISS index
                query = np.array([face_encoding])
                D, I = index.search(query, 1)

                if D[0][0] < 0.6:  # Adjust this threshold as needed
                    # Match found
                    empid = list(user_db.keys())[I[0][0]]
                    user_info = user_db[empid]
                    print(f"Authorized: {user_info['name']}, Folder: {user_info['folder']}")
                    break
                else:
                    print("Not Authorized: Face doesn't match any registered user.")
                    break
            else:
                print("No face found in the image for verification.")

    cam.release()
    cv2.destroyAllWindows()

# Example usage for verification
if __name__ == "__main__":
    verify_user_with_webcam()


Press SPACE to capture the image for verification and ESC to exit.
Not Authorized: Face doesn't match any registered user.
